In [1]:
import numpy as np
import pandas as pd
import os
import random
random.seed(1)

from tqdm import tqdm_notebook as tqdm

import lightgbm as lgb

In [2]:
input_path = '../datasets/'

In [3]:
# log_df = pd.read_csv(os.path.join(input_path, 'carlog.csv'))
meta_df = pd.read_csv(os.path.join(input_path, 'meta.csv'))
test_df = pd.read_csv(os.path.join(input_path, 'test.csv'))
product_master_df = pd.read_csv(os.path.join(input_path, 'product_master.csv'))
display_action_id_df = pd.read_csv('../datasets/display_action_id.csv')


# 特徴量の作成

In [4]:
is_predict = True

In [5]:
log_df2 = pd.read_csv('../output/cart_log.csv')
log_df2.head()

/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
0,0,2003,2019-02-14,4,0,136,bE94Gct4vGiKM7z2Y79cjk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0,2003,2019-02-14,4,32,209,bE94Gct4vGiKM7z2Y79cjk,4522646719765,おかかおにぎり ５９円,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
2,0,2003,2019-02-14,4,37,209,bE94Gct4vGiKM7z2Y79cjk,4522646695663,梅（種なし）おにぎり５９,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
3,0,2003,2019-02-14,4,52,209,bE94Gct4vGiKM7z2Y79cjk,4522646716641,紅鮭おにぎり７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0
4,0,2003,2019-02-14,4,59,209,bE94Gct4vGiKM7z2Y79cjk,4522646695670,明太子おにぎり ７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0


## クーポン発見レコードのみに絞る

In [7]:
def get_qoupon_ticketing_record(df, cols=['session_id', 'value_1']):
    df = df[df.display_action_id.isin([8, 9, 20, 28])]
    return df[cols]

qoupon_ticketing_df = get_qoupon_ticketing_record(log_df2)
qoupon_ticketing_df.head()

,session_id,value_1
1208,107,2812
2889,190,2745
3078,196,2585
3105,197,2818
7286,431,2775


## 商品マスタと紐づけてcategory_id, nameの取得

In [8]:
def join_category_id(df, product_master_df):
    df['value_1'] = df['value_1'].astype('int64')
    df = pd.merge(df, product_master_df[['JAN', 'category_id', 'category_name']], left_on='value_1', right_on='JAN', how='left')
    return df.drop(['value_1', 'JAN'], axis=1)

category_event_df = join_category_id(qoupon_ticketing_df, product_master_df)
category_event_df.head()

ValueError: cannot convert float NaN to integer

## 対象商品のみに絞る

In [23]:
categories = ['米飯(麺)__丼', '米飯(麺)__チルド米飯', 'たばこ__たばこその他', '和酒__清酒',
       '缶飲料__炭酸（缶・ボトル缶）', '米飯(麺)__原料・包材（米飯）', '水・炭酸水__小型PET（水）', '調理品__スープ・春雨', '乳製品__チーズ',
       'たばこ__輸入たばこ', 'デザート__要冷_洋生菓子', 'ハードリカー__スピリッツ',
       '調理家電__調理家電雑貨', '生活家電__生活家電雑貨', '生活家電__クリーナー', '健康・理美容家電__美容小物',
       'ハードリカー__ワイン','農産加工__水煮', '塩干(イン加工)__塩サバ', '塩干(イン加工)__干物', '冷凍__蒲焼・佃煮', '加工__冷凍(おかず)','塩干(イン加工)__鮭鱒', 
       '塩干(イン加工)__乾物', '塩干(イン加工)__漬魚', '生魚__魚惣菜', '温惣菜__コロッケ', '温惣菜__唐揚げ', '温惣菜__天ぷら', '温惣菜__フライカツ',
       '温惣菜__焼き物', '温惣菜__ＦＦ・甘味',
       '和菓子__半生・焼菓子', 'デザート__プリン', 'チョコ・ビスクラ__ビスクラ', 'デザート__ヨーグルト', '和菓子__季節(和菓子)',
       '和酒__焼酎', '調理品__和洋即席', '嗜好品__ジャム・蜂蜜',
       '衛生__ベビー用品', '珍味__珍味', '日配飲料__果汁・野菜', '乳製品__バター･マーガリン',
       'アイスクリーム__プレミアム', '麺類__パスタ', 'デザート__ゼリー', '和菓子__ゼリー','スナック・キャンディー__グミ', 'アイスクリーム__マルチ', '塩干__珍味', '塩干__魚卵',
       'スナック・キャンディー__キャンディ', '塩干__その他',
       '乳製品__ホイップ', '嗜好品__コーヒー・紅茶', '小型PET__無糖茶（小型PET）',
       '小型PET__炭酸（小型PET）',
       '調理素材__製菓材・デザート', '缶飲料__乳性（缶・ボトル缶）',
       'ハードリカー__洋酒', '冷惣菜__和惣菜',  '塩干__海藻', '塩干__ふりかけ', '塩干(イン加工)__珍味', '和洋菓子__洋菓子',
       'チョコ・ビスクラ__季節(洋菓子)', '調理品__フルーツ缶',
       '和洋菓子__和菓子', '冷凍食品__お弁当用', '冷凍食品__おかず', '和惣菜__チルド中華',
       '加工肉__お弁当', '加工肉__洋食', '塩干__小魚',
       '和日配__チルド麺', '麺類__乾麺・半生麺・棒麺',
       '農産加工__簡便野菜', '水・炭酸水__大型PET（水）',
       '珍味__豆菓子', '塩干__丸干し',
       'ホールセールパン__菓子パン(ホールセールパン)',
       '塩干__漬魚', 
       '加工肉__チルドソーセージ',
       '缶飲料__果汁（缶）', '缶飲料__紅茶（缶・ボトル缶）',
       '牛肉__挽肉', '米飯(麺)__弁当','ホールセールパン__食卓パン(ホールセールパン)',
       '小型PET__果汁（小型PET・ボトル缶）',
       '冷凍食品__麺類', '大型PET__紅茶（大型PET）',
       '水・炭酸水__缶・ボトル缶（水）',
       '米飯(麺)__おにぎり', '冷惣菜__レンジおかず', '冷惣菜__原料・包材（冷惣菜）',
       '塩干(イン加工)__季節商材',
       '羊・馬肉__ステーキ', '塩干__季節商材', 'サンド__サンド', 'ピザ__原料・包材(ピザ)',
       'サンド__OUTサンド', 'ピザ__クラスト', 'サンド__原料・包材(サンド)',
       '米飯(麺)__外食', 'サンド__サンドその他',
       'ピザ__冷凍', '明治屋テナント__ワイルドビーフ', '明治屋テナント__明治軒', '明治屋テナント__麺や天吉',
       'ピザ__チルド', '加工__おつまみ(セット）', '塩干__干物', '加工肉__ハム・ベーコン', '冷凍食品__スナック',
       '冷凍食品__米飯', '麺類__袋麺', 'キッチン__焼物・揚物',
       '缶飲料__嗜好性飲料（缶・ボトル缶）', '小型PET__紅茶（小型PET）',
       '大型PET__果汁（大型PET）', '缶飲料__スポーツ・機能性（缶・ボトル缶）',
       '大型PET__スポーツ・機能性（大型PET）', '小型PET__乳性（小型PET）', '大型PET__乳性（大型PET）',
       '缶飲料__コーヒー（ボトル缶）', '小型PET__嗜好性飲料（小型PET）', '調理家電__コーヒーメーカー',
       '珍味__小袋', '塩干__鮭鱒',
       'ホールセールパン__食パン(ホールセールパン)',
       'ホールセールパン__惣菜パン(ホールセールパン)', 'ホールセールパン__調理パン', '冷凍食品__パスタ麺',
       '加工肉__おかず', '玉子__玉子', '野菜__きのこ類', '野菜__土物類', '野菜__葉茎菜類', 
       'ホールセールパン__携帯型サンドイッチ', '100円均一__食品', '和日配__納豆', '和日配__豆腐', '和菓子__米菓', '日配飲料__果汁・野菜', '日配飲料__牛乳']

In [24]:
def get_target_category_record(df):
    return df[df['category_name'].isin(categories)]

target_category_event_df = get_target_category_record(category_event_df)
target_category_event_df.head()

,session_id,n_items,predict,category_id,category_name
0,0,1,0,617.0,米飯(麺)__おにぎり
1,0,1,0,617.0,米飯(麺)__おにぎり
2,0,1,0,617.0,米飯(麺)__おにぎり
3,0,1,0,617.0,米飯(麺)__おにぎり
4,0,1,0,617.0,米飯(麺)__おにぎり


## 予測対象の行のみに絞る

In [25]:
def get_target_category_record(df, is_predict=True):
    if is_predict:
        df = df[df['predict']==0]
    return df.drop(['predict'], axis=1)

target_row_event_df = get_target_category_record(target_category_event_df, is_predict=is_predict)
target_row_event_df.head()

,session_id,n_items,category_id,category_name
0,0,1,617.0,米飯(麺)__おにぎり
1,0,1,617.0,米飯(麺)__おにぎり
2,0,1,617.0,米飯(麺)__おにぎり
3,0,1,617.0,米飯(麺)__おにぎり
4,0,1,617.0,米飯(麺)__おにぎり


## 購入したか集計

In [26]:
def calc_is_payment(df):
    df = df.groupby(['session_id', 'category_id'])[['n_items']].sum().reset_index()
    df.loc[df['n_items'] > 0, 'n_items'] = 1
    df.loc[df['n_items'] < 0, 'n_items'] = 0
    return df
target_item_sum_df = calc_is_payment(target_row_event_df)
target_item_sum_df.head()

,session_id,category_id,n_items
0,0,617.0,1
1,2,508.0,1
2,2,617.0,1
3,3,205.0,1
4,3,365.0,1


## Pivot

In [27]:
def pivot_df(df):
    df = df.pivot(index="session_id", columns="category_id", values="n_items").reset_index()
    return df

train_payment_Y = pivot_df(target_item_sum_df)
train_payment_Y.head()

category_id,session_id,35.0,37.0,39.0,40.0,86.0,111.0,112.0,135.0,137.0,...,679.0,684.0,708.0,711.0,716.0,720.0,724.0,769.0,770.0,771.0
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Join meta.csv

In [28]:
def join_meta_df(df, meta_df):
    session_df = meta_df[['session_id']]
    df = pd.merge(session_df, df, on='session_id', how='left')
    df.fillna(0, inplace=True)
    return df

all_sessions_Y = join_meta_df(train_payment_Y, meta_df)
all_sessions_Y.head()

,session_id,35.0,37.0,39.0,40.0,86.0,111.0,112.0,135.0,137.0,...,679.0,684.0,708.0,711.0,716.0,720.0,724.0,769.0,770.0,771.0
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## カラム名を文字列方に

In [29]:
def col_name_to_str(df):
    cols = list(df.columns)
    str_cols = []
    for col in cols:
        if col == 'session_id':
            str_cols.append(col)
        else:
            str_cols.append('category_' + str(int(col)))
    df.columns = str_cols
    return df

all_sessions_Y = col_name_to_str(all_sessions_Y)
all_sessions_Y.head()

,session_id,category_35,category_37,category_39,category_40,category_86,category_111,category_112,category_135,category_137,...,category_679,category_684,category_708,category_711,category_716,category_720,category_724,category_769,category_770,category_771
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
if is_predict:
    print('is_predict')
    all_sessions_Y.to_csv('../output/given_train_category.csv', index=None)
else:
    all_sessions_Y.to_csv('../output/all_train_category.csv', index=None) # 今からパス直すの面倒なので

is_predict
